In [ ]:
# https://www.tensorflow.org/lite/tutorials/model_maker_image_classification
from __future__ import absolute_import, division
from BashColors import C
from CV2_Utils_2 import *
from TarfileFunctions import *

from time import perf_counter, sleep
import itertools, json, os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'

import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

# from tflite_support.metadata_writers import image_classifier
# from tflite_support.metadata_writers import writer_utils
import matplotlib.pyplot as plt

contentPath=os.getcwd()
image_path=os.path.join(contentPath, 'images')
generatorPath=os.path.join(contentPath, 'DataGenerator')
imagePath=join(contentPath, '3b7d7d8a64.jpg')

initialGlobList:list
with open("initialGlobList.json", 'r') as f:
    initialGlobList = json.load(f)
    
def listNewFiles(initial=initialGlobList, delete=False):
    currentFilesGlob=glob.glob('**')
    if len(initial) == len(currentFilesGlob):
        print(f'{C.BIRed}No new files in content.')
    for fil in currentFilesGlob:
        if not fil in initial:
            if isdir(fil):
                print(f'{C.BIBlue}{fil}')
                if delete:
                    shutil.rmtree(fil)
            elif isfile(fil):
                print(f'{C.ColorOff}{fil}')
                if delete:
                    os.remove(fil)
listNewFiles()

In [ ]:
data = DataLoader.from_folder(generatorPath)
train_ds, restData = data.split(0.8)
val_ds, test_ds = restData.split(0.5)
print()
print('\nnum_classes:', train_ds.num_classes)
print('class names:', train_ds.index_to_label)

print(len(train_ds) // 8, len(val_ds)//8)

In [ ]:
BATCH_SIZE = 16
EPOCHS = 1
DROPOUT_RATE = 0.2
LEARNING_RATE = 0.015
MOMENTUM = 0.95

start=perf_counter()
model = image_classifier.create(
    train_data = train_ds,
    validation_data = val_ds,
    model_spec = model_spec.get('mobilenet_v2'),
    learning_rate = LEARNING_RATE,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    dropout_rate = DROPOUT_RATE,
    use_augmentation = True, # default=False
    use_hub_library = True,  # default=True
    momentum = MOMENTUM, # Only used when use_hub_library is True
    shuffle = False,
    train_whole_model = False)

finish=perf_counter()
cvu.printTime(start, finish)

In [ ]:
loss, accuracy = model.evaluate(test_ds)
accuracy = round(accuracy, 4)
print(f'loss: {loss}\taccuracy: {accuracy*100}%')

In [ ]:
# standard code to convert to a tflite model:
converter = tf.lite.TFLiteConverter.from_saved_model(path + 'saved_model')
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8, tf.lite.OpsSet.TFLITE_BUILTINS]
tflite_model = converter.convert()

In [ ]:
 def testTFLiteModel(thisModelPath, inputImage):
    '''thisModelPath: fully qualified path to tflite model.'''
    interpreter = tf.lite.Interpreter(model_path=thisModelPath)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.allocate_tensors()
    
    input_data = np.expand_dims(new_img, axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()
help(testTFLiteModel)

In [ ]:
fileName = 'Defcon4_mobilenet_v2_Dynamic.tflite'
config = QuantizationConfig.for_dynamic(optimizations=)
help(config)

In [ ]:
model.export(export_dir=contentPath,
             tflite_filename=fileName,
             quantization_config=config)

In [ ]:
model.export(
    export_dir=contentPath, export_format=ExportFormat.LABEL)

In [ ]:
from PIL import Image
class TensorflowLiteClassificationModel:
    def __init__(self, model_path, labels, image_size=224):
        self.interpreter = tf.lite.Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()
        self._input_details = self.interpreter.get_input_details()
        self._output_details = self.interpreter.get_output_details()
        self.labels = labels
        self.image_size=image_size

    def run_from_filepath(self, image_path):
        input_data_type = self._input_details[0]["dtype"]
        image = np.array(Image.open(image_path).resize((self.image_size, self.image_size)), dtype=input_data_type)
        if input_data_type == np.float32:
            image = image / 255.

        if image.shape == (1, 224, 224):
            image = np.stack(image*3, axis=0)

        return self.run(image)

    def run(self, image):
        """
        args:
          image: a (1, image_size, image_size, 3) np.array

        Returns list of [Label, Probability], of type List<str, float>
        """

        self.interpreter.set_tensor(self._input_details[0]["index"], image)
        self.interpreter.invoke()
        tflite_interpreter_output = self.interpreter.get_tensor(self._output_details[0]["index"])
        probabilities = np.array(tflite_interpreter_output[0])

        # create list of ["label", probability], ordered descending probability
        label_to_probabilities = []
        for i, probability in enumerate(probabilities):
            label_to_probabilities.append([self.labels[i], float(probability)])
        return sorted(label_to_probabilities, key=lambda element: element[1])

In [ ]:
# Usage
modelPath='/home/jovyan/Defcon4_mobilenet_v2_ObjectClassifier.tflite'
imagePath=join(contentPath, '3b7d7d8a64.jpg')
labelsPath=join(contentPath, 'labels.txt')
model = TensorflowLiteClassificationModel(
    model_path=modelPath, labels=labelsPath)
img=cvu.getCV2Image(imagePath)
# (label, probability) = model.run_from_filepath(image_path=imagePath)

In [ ]:
# modelPath='/home/jovyan/Defcon4_mobilenet_v2_ObjectClassifier.tflite'
import tensorflow as tf
import tflite_runtime.interpreter as tflite
interpreter = tf.lite.Interpreter(model_path=modelPath)
interpreter.allocate_tensors()

# help(interpreter.get_tensor)
interpreter.get_tensor(tensor_index=177)

In [ ]:
length=len(test_ds)
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

plt.figure(figsize=(20, 20))
plt.rcParams.update({'font.size': 30})
predicts = model.predict_top_k(test_ds)
for i, (image, label) in enumerate(test_ds.gen_dataset().unbatch().take(20)):

    ax = plt.subplot(5, 4, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image.numpy(), cmap=plt.cm.gray) # .gray

    predict_label = predicts[i][0][0]
    color = get_label_color(predict_label,
                          test_ds.index_to_label[label.numpy()])
    ax.xaxis.label.set_color(color)
    plt.xlabel('Predicted: %s' % predict_label)
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf

modelPath='/home/jovyan/Defcon4_mobilenet_v2_ObjectClassifier.tflite'
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=modelPath)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)